### 3. Analysis & Insights
**Objective**
Execute comprehensive data-driven analysis using SQL to uncover actionable insights about student learning patterns, performance predictors, and at-risk student identification within the OULAD dataset.

**Analysis Framework**

This analysis employs 12 strategic queries and dashboard preparation to answer critical educational questions:

- **What drives student success vs. failure?**
- **Can we predict at-risk students early?**
- **How do demographics impact learning outcomes?**
- **Which engagement patterns lead to better performance?**

In [0]:
-- Set database context
USE eduanalytics;

-- Query 1: Overall Performance Distribution
SELECT 
    'Performance Distribution' as analysis_type,
    final_result,
    COUNT(*) as student_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) as percentage
FROM eduanalytics.v_student_analytics_master
GROUP BY final_result
ORDER BY student_count DESC;

Overall Student Performance Distribution:
> Only 1 in 3 students crosses the finish line successfully. Nearly a third simply disappear—withdrawing before they fail. 
- **Pass rate is 37.93%** - the largest single category with 12,361 students.
- **High withdrawal rate of 31.16%** (10,156 students) indicates significant attrition.
- **Failure rate is 21.64%** (7,052 students).
- Only **9.28% achieve distinction** (3,024), suggesting high academic standards. 

In [0]:
-- Query 2: Engagement vs Academic Performance
SELECT 
    'Engagement vs Performance' as analysis_type,
    engagement_level,
    final_result,
    COUNT(*) as student_count,
    ROUND(AVG(average_score), 2) as avg_assessment_score,
    ROUND(AVG(total_clicks), 2) as avg_total_clicks
FROM eduanalytics.v_student_analytics_master
WHERE final_result IS NOT NULL
GROUP BY engagement_level, final_result
ORDER BY engagement_level, final_result;

**Engagement vs Academic Performance**
> High engagement correlates with better outcomes, but the "No Engagement" group has surprisingly decent assessment scores (around 69), suggesting some students succeed despite minimal platform interaction.

**High engagement students statistics:**
- Pass rate: 76.91% vs 73.43% (low) vs 74.65% (medium)
- Assessment scores: 88.86 vs 84.28 vs 86.62
- Total clicks: 3422.95 vs 350.1 vs 738.11

In [0]:
-- Query 3: Demographics Impact Analysis
SELECT 
    'Demographics Analysis' as analysis_type,
    gender,
    age_band,
    COUNT(*) as student_count,
    ROUND(AVG(total_clicks), 2) as avg_clicks,
    ROUND(AVG(average_score), 2) as avg_score,
    ROUND(SUM(CASE WHEN final_result IN ('Pass', 'Distinction') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as success_rate
FROM eduanalytics.v_student_analytics_master
WHERE final_result IS NOT NULL AND average_score IS NOT NULL
GROUP BY gender, age_band
ORDER BY success_rate DESC;

**Demographics Impact**
> Older students (55+) achieve better outcomes with less platform activity, possibly indicating more focused study habits.
- Males aged 55+ show the highest success rate (71.26%) despite lower engagement
- Younger students (0-35) have lower success rates across both genders
- Female students aged 35-55 perform well (65.68% success rate) with moderate engagement


In [0]:
-- Query 4: Module Performance Comparison
SELECT 
    'Module Comparison' as analysis_type,
    code_module,
    COUNT(*) as total_students,
    ROUND(AVG(total_clicks), 2) as avg_clicks,
    ROUND(AVG(average_score), 2) as avg_score,
    ROUND(SUM(CASE WHEN final_result IN ('Pass', 'Distinction') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as success_rate,
    ROUND(AVG(unique_resources_accessed), 2) as avg_resources_used
FROM eduanalytics.v_student_analytics_master
WHERE final_result IS NOT NULL
GROUP BY code_module
ORDER BY success_rate DESC;


**Module Performance Comparison**
- **AAA module leads** with 70.99% success rate and second highest engagement (1666.65 clicks)
- **GGG and EEE modules** perform well (59.75% and 56.24% respectively)
- **CCC module struggles most** with only 37.84% success rate despite moderate engagement and second highest number of total students.

In [0]:
-- Query 5: Early Warning Indicators
WITH risk_categorized AS (
    SELECT 
        id_student,
        total_clicks,
        average_score,
        total_assessments_submitted,
        CASE 
            WHEN total_clicks < 100 AND average_score < 40 THEN 'High Risk'
            WHEN total_clicks < 300 OR average_score < 50 THEN 'Medium Risk'
            ELSE 'Low Risk'
        END as risk_level
    FROM eduanalytics.v_student_analytics_master
    WHERE final_result IS NOT NULL AND average_score IS NOT NULL
)
SELECT 
    'Early Warning Analysis' as analysis_type,
    risk_level,
    COUNT(id_student) as student_count,
    ROUND(AVG(total_clicks), 2) as avg_clicks,
    ROUND(AVG(average_score), 2) as avg_score,
    ROUND(AVG(total_assessments_submitted), 2) as avg_assessments_submitted
FROM risk_categorized
GROUP BY risk_level
ORDER BY 
    CASE risk_level 
        WHEN 'High Risk' THEN 1
        WHEN 'Medium Risk' THEN 2
        ELSE 3
    END;

**Early Warnings Indicator**
> Strong correlation between engagement and outcomes. High-risk students show 42x lower platform interaction (44 vs 1,887 clicks) and minimal assessment completion (1.84 vs 8.74 submissions).
- **High Risk:** 206 students (0.8%) - Critical intervention needed
- **Medium Risk:** 6,664 students (25%) - Enhanced support required
- **Low Risk:** 19,857 students (73%) - Standard monitoring

In [0]:
-- Query 6: Assessment Type Performance Analysis
WITH assessment_scores AS (
    SELECT 
        'TMA' as assessment_type,
        avg_tma_score as score,
        id_student
    FROM eduanalytics.v_student_analytics_master
    WHERE avg_tma_score IS NOT NULL
    
    UNION ALL
    
    SELECT 
        'CMA' as assessment_type,
        avg_cma_score as score,
        id_student
    FROM eduanalytics.v_student_analytics_master
    WHERE avg_cma_score IS NOT NULL
    
    UNION ALL
    
    SELECT 
        'Exam' as assessment_type,
        avg_exam_score as score,
        id_student
    FROM eduanalytics.v_student_analytics_master
    WHERE avg_exam_score IS NOT NULL
)
SELECT 
    'Assessment Type Analysis' as analysis_type,
    assessment_type,
    COUNT(*) as students_with_scores,
    ROUND(AVG(score), 2) as average_score,
    ROUND(MIN(score), 2) as min_score,
    ROUND(MAX(score), 2) as max_score,
    ROUND(STDDEV(score), 2) as score_std_dev
FROM assessment_scores
GROUP BY assessment_type
ORDER BY average_score DESC;


**Assessment Type Analysis**
> Computer-marked assessments yield the highest scores with lowest standard deviation (17.99), suggesting either more accessible content or grade inflation. Exams show the most challenging performance with highest variability (20.48 std dev), indicating authentic assessment difficulty.

1. Assessment Participation:

- **TMA (Tutor Marked Assignments):** 25,960 students (97% participation)
- **CMA (Computer Marked Assessments):** 16,944 students (63% participation)
- **Exam:** 5,998 students (22% participation)

2. Performance Hierarchy:

- **CMA:** 78% average (highest performance, lowest variability)
- **TMA:** 71.35% average (consistent performance across largest cohort)
- **Exam:** 65.01% average (lowest performance, highest variability)


In [0]:
-- Query 7: Correlation Analysis - Key Metrics
SELECT 
    'Correlation Analysis' as analysis_type,
    'Clicks vs Score' as metric_pair,
    ROUND(CORR(total_clicks, average_score), 3) as correlation_coefficient,
    COUNT(*) as sample_size
FROM eduanalytics.v_student_analytics_master
WHERE total_clicks > 0 AND average_score IS NOT NULL

UNION ALL

SELECT 
    'Correlation Analysis' as analysis_type,
    'Resources vs Score' as metric_pair,
    ROUND(CORR(unique_resources_accessed, average_score), 3) as correlation_coefficient,
    COUNT(*) as sample_size
FROM eduanalytics.v_student_analytics_master
WHERE unique_resources_accessed > 0 AND average_score IS NOT NULL

UNION ALL

SELECT 
    'Correlation Analysis' as analysis_type,
    'Engagement Span vs Score' as metric_pair,
    ROUND(CORR(engagement_span_days, average_score), 3) as correlation_coefficient,
    COUNT(*) as sample_size
FROM eduanalytics.v_student_analytics_master
WHERE engagement_span_days > 0 AND average_score IS NOT NULL;

**Correlation Analysis**
- **Engagement Span vs Score:** 0.299 (strongest predictor)
- **Clicks vs Score:** 0.286 (moderate correlation)
- **Resources vs Score:** 0.239 (weakest correlation)
> Students who maintain consistent engagement over time outperform those who simply access more resources or click frequently in short bursts. Quality of sustained interaction trumps quantity of resource consumption.

In [0]:
-- Query 8: Detailed Performance Breakdown by Education Level
SELECT 
    'Education Level Analysis' as analysis_type,
    highest_education,
    COUNT(*) as student_count,
    ROUND(AVG(total_clicks), 2) as avg_clicks,
    ROUND(AVG(average_score), 2) as avg_score,
    ROUND(AVG(unique_resources_accessed), 2) as avg_resources,
    ROUND(SUM(CASE WHEN final_result = 'Pass' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as pass_rate,
    ROUND(SUM(CASE WHEN final_result = 'Distinction' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as distinction_rate,
    ROUND(SUM(CASE WHEN final_result = 'Withdrawn' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as withdrawal_rate
FROM eduanalytics.v_student_analytics_master
WHERE final_result IS NOT NULL
GROUP BY highest_education
ORDER BY avg_score DESC;


**Education Level Analysis**
**Student Distribution (N=32,593):**
- **A Level or Equivalent:** 14,045 students (43% - largest cohort)
- **Lower Than A Level:** 13,158 students (40% - significant population)
- **HE Qualification:** 4,730 students (15% - strong performers)
- **Post Graduate:** 313 students (1% - elite group)
- **No Formal Quals:** 347 students (1% - highest risk)
> The 40% cohort with below A-level education represents the highest-volume, highest-risk population requiring targeted support interventions. Withdrawal rates inversely correlate with education level - from 23.6% (post-grad) to 42.9% (no formal quals). Students without formal qualifications are nearly twice as likely to withdraw.

In [0]:
-- Query 10: Summary Dashboard View
CREATE OR REPLACE VIEW eduanalytics.v_dashboard_summary AS
SELECT 
    -- Overall metrics
    COUNT(*) as total_students,
    COUNT(DISTINCT code_module) as total_modules,
    COUNT(DISTINCT CONCAT(code_module, '_', code_presentation)) as total_presentations,
    
    -- Performance metrics
    ROUND(AVG(average_score), 2) as overall_avg_score,
    ROUND(SUM(CASE WHEN final_result IN ('Pass', 'Distinction') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as overall_success_rate,
    
    -- Engagement metrics
    ROUND(AVG(total_clicks), 2) as avg_total_clicks,
    ROUND(AVG(unique_resources_accessed), 2) as avg_resources_accessed,
    
    -- Risk distribution
    ROUND(SUM(CASE WHEN total_clicks < 100 AND average_score < 40 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as high_risk_percentage,
    ROUND(SUM(CASE WHEN total_clicks < 300 OR average_score < 50 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as medium_risk_percentage,
    
    -- Demographics
    ROUND(SUM(CASE WHEN gender = 'F' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as female_percentage,
    ROUND(SUM(CASE WHEN disability = 'Y' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as disability_percentage,
    
    -- Engagement levels
    ROUND(SUM(CASE WHEN engagement_level = 'High' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as high_engagement_percentage,
    ROUND(SUM(CASE WHEN engagement_level = 'No Engagement' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as no_engagement_percentage
    
FROM eduanalytics.v_student_analytics_master
WHERE final_result IS NOT NULL;

-- Display the dashboard summary
SELECT * FROM eduanalytics.v_dashboard_summary;


## OULAD Dashboard: The Complete Picture

**Scale & Scope:**  
32,593 students across 7 modules and 22 presentations - representing comprehensive online learning analytics.

**Academic Performance:**
- **Average Score**: 72.8% (solid performance)
- **Success Rate**: 47.2% (concerning - less than half succeed)
- **Risk Profile**: 39% medium/high risk students need intervention

**Engagement Reality:**
- **Active Learners**: 37% show high engagement (1,215 avg clicks)
- **Digital Dropouts**: 10% show no engagement - already lost
- **Resource Utilization**: 60 resources accessed per student on average

**Demographics:**
- **Gender**: 45% female representation
- **Accessibility**: 10% students with disabilities require specialized support

**Critical Finding:**  
Despite decent academic scores (73%), the **47% success rate** reveals a completion crisis. One in ten students shows zero engagement, while 39% are at-risk - indicating systemic retention challenges in online education.

**Bottom Line:**  
Strong academic performance among engaged students, but significant attrition and engagement gaps require immediate institutional intervention strategies.

### 3. Data Mart Creation: From OLAP to Specialized Analytics

Our OLAP layer (`v_student_analytics_master`) provides comprehensive student analytics, but BI tools need **specialized, performance-optimized datasets**. The following export queries transform our unified OLAP data into focused **data marts** tailored for specific business functions:

In [0]:
-- Final data export queries for visualization tools
-- Export 1: Student performance data for dashboards
SELECT 
    code_module,
    code_presentation,
    final_result,
    gender,
    age_band,
    highest_education,
    disability,
    total_clicks,
    average_score,
    engagement_level,
    unique_resources_accessed,
    total_assessments_submitted,
    CASE 
        WHEN total_clicks < 100 AND average_score < 40 THEN 'High Risk'
        WHEN total_clicks < 300 OR average_score < 50 THEN 'Medium Risk'
        ELSE 'Low Risk'
    END as risk_level
FROM eduanalytics.v_student_analytics_master
WHERE final_result IS NOT NULL
ORDER BY code_module, code_presentation, average_score DESC;

In [0]:
-- Export 2: Module-level summary for reporting
SELECT 
    code_module,
    code_presentation,
    COUNT(*) as total_students,
    ROUND(AVG(total_clicks), 2) as avg_clicks,
    ROUND(AVG(average_score), 2) as avg_score,
    ROUND(SUM(CASE WHEN final_result IN ('Pass', 'Distinction') THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as success_rate,
    ROUND(SUM(CASE WHEN final_result = 'Withdrawn' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as withdrawal_rate,
    ROUND(AVG(unique_resources_accessed), 2) as avg_resources_used,
    COUNT(CASE WHEN engagement_level = 'High' THEN 1 END) as high_engagement_students,
    COUNT(CASE WHEN engagement_level = 'No Engagement' THEN 1 END) as no_engagement_students
FROM eduanalytics.v_student_analytics_master
WHERE final_result IS NOT NULL
GROUP BY code_module, code_presentation
ORDER BY code_module, code_presentation;

In [0]:
-- Export 3: Time-series data for trend analysis
SELECT 
    code_module,
    code_presentation,
    DATE_TRUNC('week', first_interaction_date) as week_start,
    COUNT(*) as students_starting_engagement,
    AVG(total_clicks) as avg_eventual_clicks,
    AVG(average_score) as avg_eventual_score
FROM eduanalytics.v_student_analytics_master
WHERE first_interaction_date IS NOT NULL AND final_result IS NOT NULL
GROUP BY code_module, code_presentation, DATE_TRUNC('week', first_interaction_date)
ORDER BY code_module, code_presentation, week_start;